<a href="https://colab.research.google.com/github/Benfinkels/Cross-Channel-Attribution-Analyzer-EVC-Impact-Model/blob/main/EVC_attribution_analysisV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EVC & "Ghost Traffic" Attribution Analyzer

### **The Problem: The Attribution Gap**
Standard analytics (GA4) often fail to credit video views (EVCs) because users rarely click directly from a video to the site. Instead, they view the ad and visit later via "Direct" or "Organic" search. This leaves video campaigns looking like they have low ROI, while Organic channels appear artificially inflated.

### **The Solution: Competitive Signal Unmixing**
This tool uses **Non-Negative Least Squares (NNLS)** to mathematically "unmix" your traffic spikes and determine which channel is actually echoing your video performance.

* **Competitive Modeling:** Unlike simple correlation, this model forces traffic sources (like Direct and Organic) to "compete" for credit. This prevents double-counting if multiple channels spike at the same time (solving for multicollinearity).
* **Auto-Lag Detection:** The algorithm automatically scans a range of days (0–5) to find the exact time delay between a video view and the subsequent site visit.
* **Ghost Efficiency:** Calculates a "Ghost Conversion Rate" (Coefficient) to reveal exactly how many sessions typically appear on your site for every 1 reported EVC.

### **Step 1: Upload Data**
Run the cell below to launch the **Interactive Data Loader**.

**File Requirements:**
The loader includes an **Auto-Parser** that accepts most standard formats (Wide, Long, or Pivot). You need two files:
1.  **Session File (The Effect):** A GA4 export containing `Date`, `Channel` (e.g., Session source/medium), and `Volume` (e.g., Sessions).
2.  **Target File (The Cause):** A Google Ads export containing `Date` and your trigger metric (e.g., `EVC`, `Spend`, or `Impressions`).

> **Note:** The script will automatically detect and clean date headers (pivoted data) or date rows (wide data).

In [ ]:
# ==============================================================================
# CELL 0: UPLOAD BUTTON
# ==============================================================================
from google.colab import files
import os

print(" Click below to upload your GA4 and Google Ads CSV files:")
uploaded = files.upload()

if uploaded:
    print(f"\n {len(uploaded)} file(s) uploaded successfully:")
    for fn in uploaded.keys():
        print(f"   - {fn} ({os.path.getsize(fn)/1024:.1f} KB)")
    print("\n Now run 'Step 1: Interactive Data Loader' below.")
else:
    print("\n No files were uploaded.")

In [ ]:
# =============================================================================
# CELL 1: INTERACTIVE DATA LOADER (OPTIMIZED FOR STEP 2)
# =============================================================================
import pandas as pd
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

# 1. SCAN FOR CSV FILES
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
csv_files.sort(key=lambda x: os.path.getmtime(x), reverse=True) # Newest first

if not csv_files:
    print("❌ No CSV files found. Please upload files to the folder icon on the left.")
else:
    # 2. CREATE WIDGETS
    style = {'description_width': 'initial'}
    layout = widgets.Layout(width='600px')

    dd_session_file = widgets.Dropdown(options=csv_files, description='📂 Session Data (GA4):', style=style, layout=layout)
    dd_target_file = widgets.Dropdown(options=csv_files, description='🎯 Target Data (EVC):', style=style, layout=layout)

    # Smart defaults
    for f in csv_files:
        if 'analytic' in f.lower() or 'session' in f.lower(): dd_session_file.value = f
    for f in csv_files:
        if 'evc' in f.lower() or 'ads' in f.lower() or 'target' in f.lower(): dd_target_file.value = f

    btn_load = widgets.Button(description="Load Selected Files", button_style='primary', icon='upload')
    out = widgets.Output()

    display(widgets.VBox([
        widgets.HTML("<h3>📂 Select Your Files</h3>"),
        dd_session_file, dd_target_file, btn_load, out
    ]))

    # 3. LOADING LOGIC
    def clean_duplicates(df):
        return df.loc[:, ~df.columns.duplicated()]

    def process_session_file(filename):
        """Smart parser that differentiates between types of Wide/Pivot tables."""
        raw_head = pd.read_csv(filename, nrows=5)
        cols = raw_head.columns

        # Detect likely columns
        date_col = 'Date' if 'Date' in cols else next((c for c in cols if 'date' in str(c).lower() or 'day' in str(c).lower()), None)
        sess_col = 'Sessions' if 'Sessions' in cols else next((c for c in cols if 'session' in str(c).lower() or 'user' in str(c).lower()), None)

        # CHECK 1: PIVOT FORMAT (Headers are Dates)
        try:
            sample_headers = cols[1:10]
            valid_dates = pd.to_datetime(sample_headers, errors='coerce').notna().sum()
            headers_are_dates = valid_dates > (len(sample_headers) * 0.5)
        except: headers_are_dates = False

        # --- PARSING ---
        if headers_are_dates:
            print(f"   ↳ Format: Pivot Table detected (Un-pivoting...)")
            df = pd.read_csv(filename)
            id_col = df.columns[0]
            # Melt: Turn Date Headers into a 'Date' column
            df_long = df.melt(id_vars=[id_col], var_name='Date', value_name='Sessions')
            df_long = df_long.rename(columns={id_col: 'Session source / medium'})
            return clean_duplicates(df_long) # <--- FIXED BUG HERE (Was returning 'df')

        # CHECK 2: ALREADY LONG FORMAT
        elif date_col and sess_col:
            print(f"   ↳ Format: Standard Long (Cleaning...)")
            df = pd.read_csv(filename)
            df = clean_duplicates(df)

            # Normalize names
            if date_col != 'Date': df = df.rename(columns={date_col: 'Date'})
            if sess_col != 'Sessions': df = df.rename(columns={sess_col: 'Sessions'})

            # Identify Channel Column
            reserved = ['Date', 'Sessions']
            chan_cols = [c for c in df.select_dtypes(include=['object']).columns if c not in reserved]
            if chan_cols and 'Session source / medium' not in df.columns:
                df = df.rename(columns={chan_cols[0]: 'Session source / medium'})
            return clean_duplicates(df)

        # CHECK 3: WIDE FORMAT
        elif date_col:
            print(f"   ↳ Format: Wide (Un-pivoting...)")
            df = pd.read_csv(filename)
            df = clean_duplicates(df)
            df_long = df.melt(id_vars=[date_col], var_name='Session source / medium', value_name='Sessions')
            df_long = df_long.rename(columns={date_col: 'Date'})
            return clean_duplicates(df_long)

        else:
            print(f"   ↳ Format: Unknown. Loading as-is.")
            return clean_duplicates(pd.read_csv(filename))

    def on_load_click(b):
        global df_sessions, df_evc

        with out:
            clear_output()
            s_file = dd_session_file.value
            t_file = dd_target_file.value
            print(f"🔄 Loading...")

            try:
                # 1. LOAD SESSIONS
                df_sessions = process_session_file(s_file)

                # Critical Data Type Enforcement for Step 2
                if 'Date' in df_sessions.columns:
                    df_sessions['Date'] = pd.to_datetime(df_sessions['Date'], errors='coerce')
                    df_sessions = df_sessions.dropna(subset=['Date'])

                if 'Sessions' in df_sessions.columns:
                     # Remove commas (e.g. "1,000") and force numeric
                     df_sessions['Sessions'] = pd.to_numeric(df_sessions['Sessions'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)

                print(f"   ✅ Session Data: {len(df_sessions):,} rows loaded.")

                # 2. LOAD TARGET (EVC)
                # Scan for offset headers
                raw_evc = pd.read_csv(t_file, header=None, nrows=10)
                header_row_evc = 0
                for i, row in raw_evc.iterrows():
                     if row.astype(str).str.contains('Date|Day|EVC', case=False, regex=True).any():
                        header_row_evc = i; break

                df_evc = pd.read_csv(t_file, header=header_row_evc)
                df_evc = clean_duplicates(df_evc)

                # Normalize EVC Dates
                date_col_evc = next((c for c in df_evc.columns if 'date' in str(c).lower() or 'day' in str(c).lower()), 'Date')
                df_evc = df_evc.rename(columns={date_col_evc: 'Date'})
                df_evc['Date'] = pd.to_datetime(df_evc['Date'], errors='coerce')

                print(f"   ✅ Target Data:  {len(df_evc):,} rows loaded.")
                print("\n🎉 Success! Data is ready for Step 2.")

            except Exception as e:
                print(f"\n❌ Error loading files: {e}")
                print("Tip: Ensure your CSVs have standard headers like 'Date' and 'Sessions'.")

    btn_load.on_click(on_load_click)

### **Step 2: Configure & Run Analysis**

Run the cell below to open the **Competitive Attribution Dashboard**.

**Configuration Guide:**
* **Target (EVC):** Select the "Ghost" signal you want to explain (e.g., *Engaged-View Conversions*).
* **Channel Name Col:** The dimension for your traffic sources (e.g., *Session source / medium*).
* **Scan Lag Range:** The maximum delay to test.
    * *Recommendation:* Set to **5 days**. The tool will automatically test every delay (0 to 5) and lock onto the day where the traffic pattern best matches the EVC pattern.

**Interpreting the Results Table:**
* **Attributed EVCs:** The number of conversions the model believes actually came from this channel.
* **Conversion Rate (Ghost):** The efficiency multiplier.
    * *Example:* **0.05** means it takes roughly **20 Sessions** from this channel to generate **1 EVC**.
* **Share of Explained:** The percentage of the total signal "owned" by this channel.
    * *Note:* Because this is a **competitive model**, channels fight for credit. If "Direct" and "Organic" spike at the same time, the model gives credit to the one that fits the curve best, preventing double-counting.

In [ ]:
# =============================================================================
# CELL 2: EVC "GHOST TRAFFIC" FINDER (NNLS & AUTO-LAG)
# =============================================================================

import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.optimize import nnls
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns

current_results_df = None

def find_date_column(df):
    possible_names = ['Date', 'date', 'Clean Date', 'Day', 'day', 'Time', 'Timestamp', 'Period', 'Week']
    for col in df.columns:
        if col in possible_names: return col
    for col in df.columns:
        if 'date' in col.lower(): return col
    return None

def run_dashboard():
    global current_results_df, df_sessions, df_evc

    # --- 1. CONNECT TO LOADED DATA ---
    if 'df_sessions' not in globals() or 'df_evc' not in globals() or df_sessions is None:
        print("❌ Error: Data not found. Please run 'Cell 1' first.")
        return

    # De-duplicate
    global_df_sessions = df_sessions.loc[:, ~df_sessions.columns.duplicated()].copy()
    global_df_analysis = df_evc.loc[:, ~df_evc.columns.duplicated()].copy()

    # --- 2. WIDGET SETUP ---
    style = {'description_width': 'initial'}
    layout = widgets.Layout(width='600px')

    all_cols_analysis = sorted(global_df_analysis.columns.tolist())
    all_cols_sessions = sorted(global_df_sessions.columns.tolist())
    num_cols_sessions = sorted(global_df_sessions.select_dtypes(include=[np.number]).columns.tolist())

    dd_trigger = widgets.Dropdown(options=all_cols_analysis, description='Target (EVC):', style=style, layout=layout)
    dd_channel_col = widgets.Dropdown(options=all_cols_sessions, description='Channel Name Col:', style=style, layout=layout)
    dd_value_col = widgets.Dropdown(options=num_cols_sessions, description='Traffic Metric (Sessions):', style=style, layout=layout)

    # Smart Defaults
    if 'EVC' in all_cols_analysis: dd_trigger.value = 'EVC'
    if 'Session source / medium' in all_cols_sessions: dd_channel_col.value = 'Session source / medium'
    if 'Sessions' in num_cols_sessions: dd_value_col.value = 'Sessions'

    # New: Max Lag Scanner
    slider_max_lag = widgets.IntSlider(value=5, min=1, max=14, step=1, description='Scan Lag Range (Days):', style=style, layout=layout)

    btn_run = widgets.Button(description="Run Competitive Model", button_style='primary', icon='calculator')
    btn_download = widgets.Button(description="Download Results", button_style='success', icon='download', disabled=True)
    out = widgets.Output()

    ui = widgets.VBox([
        widgets.HTML("<h3>👻 Competitive EVC Attribution (NNLS)</h3><p>Solves for multicollinearity and auto-detects time lag.</p>"),
        dd_trigger, dd_channel_col, dd_value_col, slider_max_lag,
        widgets.HBox([btn_run, btn_download]), out
    ])
    display(ui)

    # --- DOWNLOAD LOGIC ---
    def on_download_click(b):
        global current_results_df
        if current_results_df is not None:
            filename = 'EVC_Competitive_Model.csv'
            current_results_df.to_csv(filename, index=False)
            from google.colab import files
            files.download(filename)

    btn_download.on_click(on_download_click)

    # --- ANALYSIS LOGIC ---
    def on_run_click(b):
        global current_results_df
        with out:
            clear_output()
            btn_download.disabled = True

            trigger = dd_trigger.value
            channel_col = dd_channel_col.value
            value_col = dd_value_col.value
            max_lag_scan = slider_max_lag.value

            print(f"🔄 Preparing Data & Scanning Lags (0 to {max_lag_scan} days)...")

            # 1. Date Parsing
            df_s = global_df_sessions.copy()
            df_a = global_df_analysis.copy()
            date_col_s = find_date_column(df_s)
            date_col_a = find_date_column(df_a)

            if date_col_s is None or date_col_a is None:
                print("❌ CRITICAL ERROR: Could not find 'Date' column.")
                return

            df_s['Date'] = pd.to_datetime(df_s[date_col_s], errors='coerce')
            df_a['Date'] = pd.to_datetime(df_a[date_col_a], errors='coerce')
            df_a[trigger] = pd.to_numeric(df_a[trigger], errors='coerce').fillna(0)

            # 2. Pivot Sessions (The Matrix X)
            # We need a matrix where rows=Dates, Columns=Channels
            try:
                df_X = df_s.groupby(['Date', channel_col])[value_col].sum().unstack(fill_value=0)
            except Exception as e:
                print(f"❌ Error Pivoting: {e}")
                return

            # Filter out tiny channels (noise reduction)
            # Keep channels that have at least 1% of total volume or top 50
            total_vol = df_X.sum().sum()
            df_X = df_X.loc[:, df_X.sum() > (total_vol * 0.001)] # 0.1% threshold

            # 3. Align Target (The Vector y)
            df_y = df_a[['Date', trigger]].groupby('Date').sum()

            # Align Dates
            common_dates = df_X.index.intersection(df_y.index)
            if len(common_dates) < 10:
                print("❌ Not enough overlapping dates between Sessions and EVCs.")
                return

            # 4. Loop Lags to find Best Fit
            best_lag = 0
            best_score = -np.inf
            best_weights = None
            best_X = None
            best_y_aligned = None

            # Prepare Y (Target)
            # In regression: EVC(t) = Beta * Sessions(t - lag)
            # So if Lag=2, Today's EVC is explained by Sessions from 2 days ago.

            for lag in range(max_lag_scan + 1):
                # Shift X (Sessions) forward by lag to align with future EVCs
                # e.g. Session on Jan 1 (index) -> Becomes feature for Jan 3 (shifted index)

                # We simply shift the TARGET (EVC) backwards to match sessions?
                # No, standard is: align indices.
                # If lag=2, EVC at T is matched with X at T-2.

                X_aligned = df_X.shift(lag).dropna() # Sessions shifted forward
                y_aligned = df_y.loc[X_aligned.index] # Match dates

                # Clean NaNs caused by shift
                valid_idx = X_aligned.index.intersection(y_aligned.index)
                X_final = X_aligned.loc[valid_idx]
                y_final = y_aligned.loc[valid_idx][trigger].values

                if len(y_final) < 5: continue

                # Run NNLS (Non-Negative Least Squares)
                # Solves argmin_x || Ax - b ||_2 for x>=0
                weights, rss = nnls(X_final.values, y_final)

                # Calculate R2 manually for NNLS
                y_pred = np.dot(X_final.values, weights)
                score = r2_score(y_final, y_pred)

                if score > best_score:
                    best_score = score
                    best_lag = lag
                    best_weights = weights
                    best_X = X_final
                    best_y_aligned = y_final

            if best_weights is None or best_score < 0:
                print("⚠️ No significant correlation found. Check data volume.")
                return

            # 5. Compile Results
            print(f"✅ Best Fit Found! Lag: {best_lag} days (R²: {best_score:.3f})")

            channel_names = best_X.columns
            total_attributed_evc = 0

            results = []
            for i, channel in enumerate(channel_names):
                coeff = best_weights[i] # EVCs per Session
                if coeff > 0:
                    # Total Sessions in the model window
                    total_sessions_channel = best_X[channel].sum()

                    # Attributed EVCs = Coefficient * Volume
                    attr_evc = coeff * total_sessions_channel
                    total_attributed_evc += attr_evc

                    results.append({
                        'Channel': channel,
                        'Conversion Rate (Ghost)': coeff, # The "Beta"
                        'Attributed EVCs': attr_evc,
                        'Raw Sessions': total_sessions_channel
                    })

            res_df = pd.DataFrame(results).sort_values(by='Attributed EVCs', ascending=False)

            # Normalize to Observed EVCs?
            # NNLS attempts to fit the curve magnitude.
            # If under-predicting, it means signal is missing. If over, it's noise.
            # We usually display the Raw Model Output.

            res_df['Share of Explained EVCs'] = (res_df['Attributed EVCs'] / res_df['Attributed EVCs'].sum()) * 100

            current_results_df = res_df
            btn_download.disabled = False

            # 6. Display
            print("\n" + "="*80)
            print(f"🏆 ATTRIBUTION RESULT (Based on {len(best_y_aligned)} days of data)")
            print("="*80)

            styled = res_df.style.format({
                'Conversion Rate (Ghost)': '{:.5f}',
                'Attributed EVCs': '{:,.1f}',
                'Raw Sessions': '{:,.0f}',
                'Share of Explained EVCs': '{:.1f}%'
            }).background_gradient(subset=['Attributed EVCs'], cmap='Greens')

            display(styled)

            # 7. Visualization: Predicted vs Actual
            plt.figure(figsize=(12, 5))
            y_pred_best = np.dot(best_X.values, best_weights)
            plt.plot(best_X.index, best_y_aligned, label='Actual Google EVCs', color='black', alpha=0.6)
            plt.plot(best_X.index, y_pred_best, label='Model Predicted (From Traffic)', color='green', linestyle='--')
            plt.title(f'Model Fit: Actual EVCs vs Traffic Signals (Lag: {best_lag} days)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()

    btn_run.on_click(on_run_click)

run_dashboard()

---
### **⚠️ Note on Methodology**
* **Correlation vs. Causation:** While a low P-value ($<0.05$) indicates a strong statistical link, it does not prove absolute causality. Seasonality or concurrent media events can influence these numbers.
* **Directional Signal:** Use these results as a **directional signal** to identify which organic channels are absorbing your paid video demand.

### **Step 3: Validate the Model**
Run the cell below to visualize the accuracy of your attribution.

**The Charts Explained:**
* **Reality Check (Top):** Compares the **Actual EVCs** (Black Line) vs. the **Predicted Model** (Green Dashed).
    * *Goal:* You want these lines to move together. If the Green line spikes when the Black line spikes, the model works.
* **Attribution Stack (Middle):** Breaks down the Green line to show you *which channels* are driving that prediction.
    * *Use Case:* Use this to prove to stakeholders: *"See that spike on the 15th? That wasn't random—that was Organic Search echoing our video campaign."*

In [ ]:
# =============================================================================
# CELL 3: ATTRIBUTION VISUALIZER (ROBUST MATCHING)
# =============================================================================

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns
import os

def run_visualizer():
    global current_results_df, df_sessions, df_evc

    # --- 1. LOAD MODEL ---
    target_file = 'EVC_Competitive_Model.csv'
    if os.path.exists(target_file):
        current_results_df = pd.read_csv(target_file)

    if 'current_results_df' not in globals():
        print("❌ Error: Model results not found.")
        return

    # --- 2. EXTRACT WEIGHTS ---
    try:
        # Standardize Model Channels to Lowercase for better matching
        current_results_df['Channel_Clean'] = current_results_df['Channel'].astype(str).str.lower().str.strip()

        model_weights = dict(zip(current_results_df['Channel_Clean'], current_results_df['Conversion Rate (Ghost)']))
        share_map = dict(zip(current_results_df['Channel_Clean'], current_results_df.get('Share of Explained EVCs', [100]*len(model_weights))))
        active_channels = {k: v for k, v in model_weights.items() if v > 0}

        # Keep map of Clean -> Original Name for legend
        name_map = dict(zip(current_results_df['Channel_Clean'], current_results_df['Channel']))

    except Exception as e:
        print(f"❌ Error reading model: {e}")
        return

    # --- 3. WIDGET SETUP ---
    style = {'description_width': 'initial'}
    layout = widgets.Layout(width='600px')

    # A. Target (EVC)
    evc_col_guess = 'EVC'
    if 'df_evc' in globals():
        for c in df_evc.select_dtypes(include=[np.number]).columns:
            if 1000 < df_evc[c].sum() < 20000: evc_col_guess = c; break
        evc_options = sorted(df_evc.select_dtypes(include=[np.number]).columns)
    else:
        print("❌ Error: Raw EVC data missing.")
        return

    dd_vis_target = widgets.Dropdown(options=evc_options, value=evc_col_guess, description='Target (EVC):', style=style, layout=layout)

    # B. Input (Sessions)
    sess_col_guess = 'Sessions'
    if 'df_sessions' in globals():
        num_sess = df_sessions.select_dtypes(include=[np.number]).columns
        for c in num_sess:
            if c.lower() in ['sessions', 'users', 'clicks', 'traffic']: sess_col_guess = c; break
        sess_options = sorted(num_sess)

        # C. Channel Column (CRITICAL NEW SELECTOR)
        cat_sess = df_sessions.select_dtypes(exclude=[np.number]).columns
        chan_col_guess = cat_sess[0] if len(cat_sess) > 0 else None
        for c in cat_sess:
            if 'source' in c.lower() or 'medium' in c.lower() or 'channel' in c.lower(): chan_col_guess = c; break
        chan_options = sorted(cat_sess)
    else:
        print("❌ Error: Raw Session data missing.")
        return

    dd_vis_metric = widgets.Dropdown(options=sess_options, value=sess_col_guess, description='Metric (Sessions):', style=style, layout=layout)
    dd_vis_channel = widgets.Dropdown(options=chan_options, value=chan_col_guess, description='Channel Name Col:', style=style, layout=layout)

    # D. Other Settings
    slider_vis_lag = widgets.IntSlider(value=0, min=0, max=14, step=1, description='Model Lag (Days):', style=style, layout=layout)
    toggle_smooth = widgets.Checkbox(value=True, description='Smooth Data (7-Day Avg)', style=style)

    btn_viz = widgets.Button(description="Run Analysis", button_style='primary', icon='check')
    out_viz = widgets.Output()

    ui = widgets.VBox([
        widgets.HTML("<h3>⚖️ Attribution Visualizer (Robust Match)</h3>"),
        dd_vis_target, dd_vis_metric, dd_vis_channel, slider_vis_lag, toggle_smooth, btn_viz, out_viz
    ])
    display(ui)

    def on_viz_click(b):
        with out_viz:
            clear_output()
            trigger = dd_vis_target.value
            metric = dd_vis_metric.value
            chan_col = dd_vis_channel.value
            lag = slider_vis_lag.value
            smooth = toggle_smooth.value

            print(f"⚙️ Matching '{chan_col}' to Model...")

            # --- A. PREPARE PREDICTIONS (THE STACK) ---
            # 1. Clean & Pivot
            # Create a lowercase version of the channel column for matching
            df_work = df_sessions.copy()
            df_work['Channel_Join'] = df_work[chan_col].astype(str).str.lower().str.strip()

            # Pivot on the CLEAN name
            df_pivot = df_work.pivot_table(index='Date', columns='Channel_Join', values=metric, aggfunc='sum').fillna(0)
            df_X = df_pivot.shift(lag).dropna()

            # 2. Build Stack
            df_plot = pd.DataFrame(index=df_X.index)
            stack_cols = []

            # Find which model keys exist in the data
            found_channels = [c for c in active_channels.keys() if c in df_X.columns]

            if not found_channels:
                print("❌ ERROR: No matching channels found!")
                print(f"   Model expects: {list(active_channels.keys())[:3]}...")
                print(f"   Data contains: {list(df_X.columns)[:3]}...")
                print("   👉 Check your 'Channel Name Col' selection.")
                return

            # Separate Major vs Minor based on share
            major_drivers = [k for k in found_channels if share_map.get(k, 0) > 5.0]
            minor_drivers = [k for k in found_channels if share_map.get(k, 0) <= 5.0]

            for channel in major_drivers:
                clean_name = name_map.get(channel, channel).split(' / ')[0] # Use original name for display
                df_plot[clean_name] = df_X[channel] * model_weights[channel]
                stack_cols.append(clean_name)

            df_plot['Other Sources'] = 0
            has_minor = False
            for channel in minor_drivers:
                df_plot['Other Sources'] += df_X[channel] * model_weights[channel]
                has_minor = True

            if has_minor and df_plot['Other Sources'].sum() > 0:
                stack_cols.append('Other Sources')

            if not stack_cols:
                print("❌ Error: Stack columns empty. Check data volume.")
                return

            # --- B. PREPARE ACTUALS (THE LINE) ---
            df_y = df_evc.groupby('Date')[[trigger]].sum().rename(columns={trigger: 'Actual EVCs'})

            # --- C. PLOT ---
            df_final = pd.merge(df_y, df_plot, left_index=True, right_index=True, how='inner')

            if smooth:
                df_final = df_final.rolling(window=7, min_periods=1).mean()

            if df_final.empty:
                print("❌ No overlapping data dates.")
                return

            fig, ax = plt.subplots(figsize=(14, 7))

            # Use Tab20 for more colors
            colors = sns.color_palette("tab20", len(stack_cols))

            try:
                ax.stackplot(df_final.index, df_final[stack_cols].T, labels=stack_cols, colors=colors, alpha=0.85)
            except Exception as e:
                print(f"❌ Plot Error: {e}")
                return

            sns.lineplot(data=df_final, x=df_final.index, y='Actual EVCs', ax=ax,
                         color='black', linewidth=3, label='Actual Reported EVCs')

            ax.set_title(f"Attribution Stack: {trigger}", fontsize=16, fontweight='bold')
            ax.set_ylabel("Daily Conversions")
            ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), title="Source")
            ax.grid(True, alpha=0.2)
            plt.tight_layout()
            plt.show()

            # Print Stat
            total_model = df_final[stack_cols].sum().sum()
            total_actual = df_final['Actual EVCs'].sum()
            print(f"✅ Prediction: {total_model:,.0f} | Actual: {total_actual:,.0f} ({total_model/total_actual:.1%})")

    btn_viz.on_click(on_viz_click)

run_visualizer()

In [ ]:
# =============================================================================
# CELL 5: GHOST ROI CALCULATOR
# =============================================================================

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

def run_roi_calculator():
    # --- 1. LOAD MODEL ---
    target_file = 'EVC_Competitive_Model.csv'
    if os.path.exists(target_file):
        df_model = pd.read_csv(target_file)
    else:
        print("❌ Error: 'EVC_Competitive_Model.csv' not found. Run Step 2 first.")
        return

    # --- 2. WIDGET SETUP ---
    style = {'description_width': 'initial'}
    layout = widgets.Layout(width='400px')

    # Financial Inputs
    txt_aov = widgets.FloatText(value=50.00, description='Average Order Value ($):', style=style, layout=layout)
    txt_cpa = widgets.FloatText(value=15.00, description='Target CPA ($):', style=style, layout=layout)

    btn_calc = widgets.Button(description="Calculate ROI", button_style='success', icon='dollar-sign')
    out_calc = widgets.Output()

    ui = widgets.VBox([
        widgets.HTML("<h3>💰 Ghost ROI Calculator</h3><p>Translate 'Attributed Conversions' into Revenue and Media Value.</p>"),
        txt_aov, txt_cpa, btn_calc, out_calc
    ])
    display(ui)

    def on_calc_click(b):
        with out_calc:
            clear_output()
            aov = txt_aov.value
            cpa = txt_cpa.value

            # --- CALCULATIONS ---
            # Revenue = EVCs * AOV
            # Media Value = EVCs * CPA (How much you would have paid to get these elsewhere)

            df_roi = df_model.copy()
            df_roi['Ghost Revenue'] = df_roi['Attributed EVCs'] * aov
            df_roi['Media Value Created'] = df_roi['Attributed EVCs'] * cpa

            # Totals
            total_evc = df_roi['Attributed EVCs'].sum()
            total_rev = df_roi['Ghost Revenue'].sum()
            total_val = df_roi['Media Value Created'].sum()

            # --- OUTPUT ---
            print("\n" + "="*60)
            print(f"💸 FINANCIAL IMPACT ANALYSIS")
            print("="*60)
            print(f"• Total Attributed EVCs:    {total_evc:,.0f}")
            print(f"• Total Ghost Revenue:      ${total_rev:,.2f}  (Based on ${aov} AOV)")
            print(f"• Total Media Value:        ${total_val:,.2f}  (Based on ${cpa} CPA)")
            print("-" * 60)
            print(f"💡 INSIGHT: Your YouTube campaigns generated ${total_rev:,.0f} in revenue")
            print(f"   that was incorrectly attributed to other channels in GA4.")
            print("="*60 + "\n")

            # Pretty Table
            cols_to_show = ['Channel', 'Attributed EVCs', 'Ghost Revenue', 'Media Value Created']

            styled = df_roi[cols_to_show].head(10).style.format({
                'Attributed EVCs': '{:,.1f}',
                'Ghost Revenue': '${:,.2f}',
                'Media Value Created': '${:,.2f}'
            }).background_gradient(subset=['Ghost Revenue'], cmap='Greens')

            display(styled)

    btn_calc.on_click(on_calc_click)

run_roi_calculator()